In [64]:
import numpy as np
import math 
import queue

In [73]:
class MapManager:
    def __init__(self):
        self.Map = Map_init()
        self.Team1 = self.Team()
        self.Team2 = self.Team()
    
    directions = [(1, 0), (0, 1), (-1, 0), (0, -1)]
    
    class Tile:
        def __init__(self, pos, unit):
            self.pos = pos
            self.unit_ref = unit
            self.tile_cost = 1.0 #Cost to move onto this tile
            
            self.move_cost = float('inf')
            self.move_parent = None
            self.visited = False
            self.is_attack = False

        def __str__(self):
            return "Tile ({0}, {1}):\nUnit: {2}".format(self.pos[0], self.pos[1], self.unit)
        
    def Map_init():
        Map = np.empty([5, 5], dtype=Tile)
        for i in range(Map.shape[0]):
            for j in range(Map.shape[1]):
                Map[i,j] = Tile((i, j), None)
        return Map
    
    class Unit:
        def __init__(self, pos, cmove, hp, mmove, Att, Def, Team):
            self.pos = pos
            self.curr_move = cmove
            self.hp = hp
            
            self.max_move = mmove
            self.Att = att
            self.Def = Def
            self.Team = Team
            
    def find_movement(unit):
        dijstrkas(unit)
    
    def dijstrkas(unit):
        visited_tiles = []
        prio_queue = queue.PriorityQueue()
        Map = self.Map
        
        prio_queue.put(0, Map[unit.pos])
        visited_tiles.append(Map[unit.pos])
        
        while(not prio_queue.empty()):
            curr_tile = prio_queue.get()
            curr_pos = curr_tile.pos
            curr_tile.visited = True
            for i in range(4):
                new_pos = curr_pos + directions[i] 
                if (new_pos[0] < 0 or new_pos[0] >= self.Map.shape[0] or
                    new_pos[1] < 0 or new_pos[1] >= self.Map.shape[1] or
                    (Map[new_pos].unit is not None and Map[new_pos].unit.Team == unit.Team) or
                    new_pos == unit.pos or Map[new_pos].visited):
                    pass
                else:
                    new_cost = Map[new_pos].tile_cost + curr_tile.move_cost
                    if (Map[new_pos].move_cost > new_cost and new_cost < unit.curr_move):
                        Map[new_pos].move_cost = new_cost
                        Map[new_pos].move_parent = curr_tile
                        if (Map[new_pos].unit is not None):
                            Map[new_pos].is_attack = True
                        visited_tiles.append(Map[new_pos])
                        prio_queue.put(new_cost, Map[new_pos])
                    
        
        
    def clear_movement():
        for i in range(self.Map.shape[0]):
            for j in range(self.Map.shape[1]):
                self.Map[i,j].move_cost = float('inf')
                self.Map[i,j].visited = False
                self.Map[i,j].is_attack = False
                
        
    
    def move_unit(unit, pos):
        assert (self.Map[pos].unit_ref is None)
        unit.pos = pos
        self.Map[pos].unit_ref = unit
        
    
    class Team:
        def __init__(self):
            self.units = []
    

In [60]:
manager = MapManager()

NameError: name 'team' is not defined

In [61]:
Map = np.empty([5, 5], dtype=Tile)
for i in range(Map.shape[0]):
    for j in range(Map.shape[1]):
        Map[i,j] = MapManager.Tile((i, j), None)